# Predição do Estado de uma Smart Grid

# Domínio da Aplicação

## Exploração da Base de Dados

### Coletando os [dados simulados de estabilidade de uma rede elétrica](https://archive.ics.uci.edu/dataset/471/electrical+grid+stability+simulated+data)

In [ ]:
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo 

electrical_grid_stability_simulated_data = fetch_ucirepo(id=471).data

features = electrical_grid_stability_simulated_data.features 
targets = electrical_grid_stability_simulated_data.targets

data = pd.merge(features, targets, left_index=True, right_index=True, how='outer')

### Explicando os Atributos

- tau[x]: Tempo de reação do participante (valor real no intervalo [0.5, 10] segundos). 
    - tau1 - o valor para o produtor de eletricidade;
- p[x]: Potência nominal consumida (negativa) ou produzida (positiva) (valor real). 
    - Para consumidores, no intervalo [-0.5, -2] segundos^-2;
    - p1 = abs(p2 + p3 + p4);
- g[x]: Coeficiente (gamma) proporcional à elasticidade de preço (valor real no intervalo [0.05, 1] segundos^-1). 
    - g1 - o valor para o produtor de eletricidade;
- stab: A parte real máxima da raiz da equação característica (se positiva - o sistema é linearmente instável) (valor real).
- stabf: A classificação de estabilidade do sistema (categórica: estável/instável).


### Visualização os Dados

In [ ]:
data.head()

# Pré-Processamento dos Dados

### Eliminação do dado discreto, `stabf`

Com o objetivo de prever o próximo estado da rede, a partir do valor contínuo `stab`, temos que eliminar o valor `stabf` já que depende de `stab`.

In [ ]:
data.drop(columns=['stabf'], inplace=True)

### Análise das Correlações

Analisar as correlações das váriaveis para escolher as melhores váriaveis para os modelos de regressão.

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

correlation_matrix = data.corr()

plt.figure(figsize=(16, 9))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Mapa de Calor')
plt.show()

## Limpeza os dados

Vendo a baixa correlação das variáveis $p_x$ com $stab$, é decidido a remoção desses valores, para melhorar o aprendizado do modelo.

In [ ]:
data.drop(columns=['p1', 'p2', 'p3', 'p4'], inplace=True)

# Reconhecimento de Padrões e Aprendizados

### Definição de Métodos e Classes de Apoio

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def create_data_structure():
    '''
    Cria um objeto com a estrutura de dados preparada para armazenar as informações durante os treinamentos.
    '''
    return {
        "r2": [],
        "mse": [],
        "mae": [],
        "pred": [],
        "test": [],
        "mape": [],
    }

def get_class_name(object):
    return object.__class__.__name__

class Fitter:
    '''
    Classe de apoio para facilitar no treinamento de cada modelo.
    '''
    def __init__(self, x_train, y_train, x_test, y_test) -> None:
        self.__x_train = x_train
        self.__y_train = y_train
        self.__x_test = x_test
        self.__y_test = y_test

    def fit_model_and_measure(self, model, data):
        model.fit(self.__x_train, self.__y_train)
        
        pred = model.predict(self.__x_test)

        data['pred'].append(pred)
        data['test'].append(self.__y_test)
        data['r2'].append(r2_score(self.__y_test, pred))
        data['mse'].append(mean_squared_error(self.__y_test, pred))
        data['mae'].append(mean_absolute_error(self.__y_test, pred))
        data['mape'].append(np.mean(np.abs((self.__y_test - pred) / self.__y_test)) * 100)

### Definição dos modelos a serem usados.

In [ ]:
from sklearn.svm import SVR, NuSVR
from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge, ElasticNet, BayesianRidge, ARDRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor


MODELS = [
    BayesianRidge(),
    ARDRegression(),
    Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    LinearRegression(),
    SVR(kernel='linear', C=1.0),
    NuSVR(nu=0.5, kernel='rbf'),
    LGBMRegressor(max_depth=10, verbose=-1),
    KNeighborsRegressor(n_neighbors=5),
    ElasticNet(alpha=0.1, l1_ratio=0.5),
    AdaBoostRegressor(n_estimators=100),
    SGDRegressor(max_iter=1000, tol=1e-3),
    ExtraTreesRegressor(n_estimators=100),
    XGBRegressor(learning_rate=0.1, max_depth=8), 
    MLPRegressor(hidden_layer_sizes=(100,), max_iter=500),
    RandomForestRegressor(n_estimators=100, random_state=0),
    HistGradientBoostingRegressor(max_iter=100, learning_rate=0.1),
    GradientBoostingRegressor(n_estimators=100, learning_rate=0.1),
    DecisionTreeRegressor(max_depth=5, splitter='random', min_samples_leaf=5),
]

## Treinamento e Validação dos Primeiros Modelos

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import KFold

outputs = {}

for model in MODELS:
    outputs[get_class_name(model)] = create_data_structure()
 
five_folds = KFold(n_splits=5, random_state=0, shuffle=True)

STAB_COLUMN_INDEX = list(data.columns).index('stab')

number_of_operations = len(MODELS) * 5 

progress_bar = tqdm(total=number_of_operations)

y_data = data['stab'].values
x_data = data.drop(columns=['stab']).values

for index, (train_index, test_index) in enumerate(five_folds.split(data)):
    y_train = y_data[train_index]
    x_train = x_data[train_index]

    y_test = y_data[test_index]
    x_test = x_data[test_index]

    fitter = Fitter(
        x_train=x_train,
        x_test=x_test,
        y_train=y_train,
        y_test=y_test,
    )

    for model in MODELS:
        fitter.fit_model_and_measure(model, data=outputs[get_class_name(model)])
        progress_bar.update()

# Pós-Processamento e Conclusão

## Definindo funções de apoio

In [ ]:
def create_metrics(error_structure):
    metrics = {
        'MSE': error_structure['mse'], 
        'MAE': error_structure['mae'], 
        'MAPE': error_structure['mape'],
        'R2': error_structure['r2']
    }

    mean_std = {metric: (np.mean(scores), np.std(scores)) for metric, scores in metrics.items()}

    return mean_std

In [ ]:
METRICS = ['MSE', 'MAE', 'MAPE', 'R2']

model_metrics = {}

for model in MODELS:
    model_metrics[get_class_name(model)] = create_metrics(outputs[get_class_name(model)])

# Função para plotar gráficos de métricas
def plot_metrics(metric_name, model_metrics:dict, reverse=False):
    width = 0.2

    fig, ax = plt.subplots(figsize=(16, 9))

    sorted_metrics = sorted(model_metrics.items(), key=lambda x:x[1][metric_name][0], reverse=reverse)

    for index, (model_name, metric_values) in enumerate(sorted_metrics):
        x_position = index * width

        height = metric_values[metric_name][0] 

        y_error = metric_values[metric_name][1]

        ax.bar(x_position, height, width, label=model_name, yerr=y_error)

    ax.set_ylabel('Scores')
    ax.set_title(f'{metric_name} Scores by Model')
    ax.set_xticklabels("")
    ax.legend()

    plt.show()

for metric in METRICS:
    plot_metrics(metric, model_metrics, metric == 'R2')

### Filtragem dos Modelos

A partir dessa primeira comparação, será escolhido os modelos
- NuSVR
- LGBMRegressor
- HistGradientBoostRegressor
- XGBoost
- ExtraTreeRegressor

E para modelo, aplicar o método de GridSearch para otimizar o modelo sobre o problema

In [ ]:
BEST_MODELS = [
    (NuSVR(), {
        'nu': [0.1, 0.5, 0.9],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'coef0': [0.1, 0.5, 0.9],
        'shrinking': [True, False]
    }),
    (XGBRegressor(), {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.3],
        'subsample': [0.6, 0.8, 1.0]
    }), 
    (LGBMRegressor(), {
        'boosting_type': ['gbdt', 'dart', 'rf'],
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 0.3],
        'verbosity': [-1]
    })
]

In [ ]:
from os import cpu_count
from IPython.display import clear_output
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, train_test_split

number_of_cpus = cpu_count()

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

y_data = data['stab'].values
x_data = data.drop(columns=['stab']).values

fitter = Fitter(
    x_train=x_train,
    x_test=x_test,
    y_train=y_train,
    y_test=y_test,
)

best_outputs = {}

for model, _ in BEST_MODELS:
    best_outputs[get_class_name(model)] = create_data_structure()

for model, params in BEST_MODELS:
    model_name = get_class_name(model)

    grid_search = HalvingGridSearchCV(model, params, n_jobs=number_of_cpus, cv=2, scoring='r2') #['r2', 'neg_median_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error']

    fitter.fit_model_and_measure(grid_search, data=best_outputs[get_class_name(model)])

    best_outputs[model_name]['best_model'] = grid_search.best_params_

clear_output()

In [ ]:
METRICS = ['MSE', 'MAE', 'MAPE', 'R2']

model_metrics = {}

for model,_ in BEST_MODELS:
    print(model)
    print(best_outputs[get_class_name(model)]['best_model'])
    model_metrics[get_class_name(model)] = create_metrics(best_outputs[get_class_name(model)])

# Função para plotar gráficos de métricas
def plot_metrics(metric_name, model_metrics:dict, reverse=False):
    width = 0.2

    fig, ax = plt.subplots(figsize=(16, 9))

    sorted_metrics = sorted(model_metrics.items(), key=lambda x:x[1][metric_name][0], reverse=reverse)

    for index, (model_name, metric_values) in enumerate(sorted_metrics):
        x_position = index * width

        height = metric_values[metric_name][0] 

        y_error = metric_values[metric_name][1]

        ax.bar(x_position, height, width, label=model_name, yerr=y_error)

    ax.set_ylabel('Scores')
    ax.set_title(f'{metric_name} Scores by Model')
    ax.set_xticklabels("")
    ax.legend()

    plt.show()

for metric in METRICS:
    plot_metrics(metric, model_metrics, metric == 'R2')

In [ ]:
from sklearn.model_selection import GridSearchCV

fitter = Fitter(
    x_train=x_train,
    x_test=x_test,
    y_train=y_train,
    y_test=y_test,
)

params = {
    'nu': [0.9],
}

grid_search = GridSearchCV(NuSVR(), params, n_jobs=number_of_cpus, cv=5, scoring='r2', verbose=1)

results = create_data_structure()

fitter.fit_model_and_measure(grid_search, results)

print(grid_search.best_params_)

print(results)

# Conclusão

Responder à pergunta: "Você acredita que será possível entregar tudo que prometeu no documento da Proposta?"


Sim, por mais que esteja apenas no começo do projeto, já é possivel fazer algumas análises utilizando regressão linear e arvore de regressão como visto em aula, com o que já temos fica facil crial pipelines para os ajustes nos dados e também abre a possibilidade para random forests. Com o objetivo final de evitar overfitting e garantir também que seja possivel obter respostas e tomadas de decisões razoáveis em relação aos dados estabelecidos.